<a href="https://colab.research.google.com/github/Srivastava0109Yash/-PCS24-67/blob/main/mistral_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone 'https://github.com/ali7919/Enlighten-Instruct.git'
!pip install -U bitsandbytes
!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer,PPOTrainer,DPOTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re,json

In [3]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=512,
                                 do_sample=True,
                                 pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [52]:
mydata=load_dataset(path='/content/',data_files='dataset.jsonl')
a=mydata["train"]
a

Dataset({
    features: ['messages'],
    num_rows: 11
})

In [48]:
for i in a:
  print(i)

{'messages': [{'role': 'system', 'content': '[{"byFunc": "onchaintart", "input": "tell me the current account payable for advanced micro Devices for 2022", "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "AgentExecutor", "type": "custom_agent"}]'}, {'role': 'user', 'content': '[]'}, {'role': 'assistant', 'content': '[]'}]}
{'messages': [{'role': 'system', 'content': '[]'}, {'role': 'user', 'content': '[{"input": "tell me the current account payable for advanced micro Devices for 2022", "output": null, "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "LLMChain", "type": "custom_chain"}]'}, {'role': 'assistant', 'content': '[]'}]}
{'messages': [{'role': 'system', 'content': '[{"byFunc": "onllmstart", "input": "[\\"Answer the following questions as best you can. You have access to the following tools:\\\\n\\\\nSearch Engine RAG: use this as the first tool to answer query, dont use any other tool .This tool extracts data from database. If you dont g

In [45]:
for i in range(len(a)):
  for j in range(3):
    print(mydata["train"]['messages'][i][j])




{'role': 'system', 'content': '[{"byFunc": "onchaintart", "input": "tell me the current account payable for advanced micro Devices for 2022", "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "AgentExecutor", "type": "custom_agent"}]'}
{'role': 'user', 'content': '[]'}
{'role': 'assistant', 'content': '[]'}
{'role': 'system', 'content': '[]'}
{'role': 'user', 'content': '[{"input": "tell me the current account payable for advanced micro Devices for 2022", "output": null, "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "LLMChain", "type": "custom_chain"}]'}
{'role': 'assistant', 'content': '[]'}
{'role': 'system', 'content': '[{"byFunc": "onllmstart", "input": "[\\"Answer the following questions as best you can. You have access to the following tools:\\\\n\\\\nSearch Engine RAG: use this as the first tool to answer query, dont use any other tool .This tool extracts data from database. If you dont get answer through this just reply \\\\\\"Data Not 

In [31]:
main_dict = {}

for item in a:
    role = item['role']
    content = json.loads(item['content'])
    main_dict[role] = content

print(main_dict)


{'system': [{'byFunc': 'onchaintart', 'input': 'tell me the current account payable for advanced micro Devices for 2022', 'operatorDescriptiveFeedback': '', 'operatorFeedback': '-1', 'name': 'AgentExecutor', 'type': 'custom_agent'}], 'user': [], 'assistant': []}


In [24]:
type(a)

list

In [133]:
len(a)

2

In [132]:
a

{'role': 'assistant', 'content': '[]'}

In [114]:
type(mydata)

datasets.dataset_dict.DatasetDict

In [126]:
type(a)

dict

In [55]:
# import json
# def create_collective_prompt(data):
#     prompts = []

#     def create_prompt(sample):
#         bos_token = "<s>"
#         system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
#         response = sample[0]
#         input_text = sample["input"]
#         eos_token = "</s>"

#         full_prompt = ""
#         full_prompt += bos_token
#         full_prompt += "### Instruction:"
#         full_prompt += "\n" + system_message
#         full_prompt += "\n\n### Input:"
#         full_prompt += "\n" + input_text
#         full_prompt += "\n\n### Response:"
#         full_prompt += "\n" + response
#         full_prompt += eos_token

#         return full_prompt

#     # Iterate through the data and apply the create_prompt function
#     for sample in data:
#         messages = sample.get("messages", [])
#         input_message = next((msg["content"] for msg in messages if msg["role"] == "user"), None)
#         response_message = next((msg["content"] for msg in messages if msg["role"] == "assistant"), None)

#         if input_message and response_message:
#             sample["input"] = json.loads(input_message)[0].get("input", "")
#             sample[0] = json.loads(response_message)[0].get("output", "")

#             prompt = create_prompt(sample)
#             prompts.append(prompt)

#     return '\n'.join(prompts)


def create_prompt(sample):
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = str(sample["messages"])
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += system_message
  full_prompt += "[/INST]"
  full_prompt += response
  full_prompt += eos_token

  return full_prompt

In [56]:
create_prompt(a)

'<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.[/INST][[{\'role\': \'system\', \'content\': \'[{"byFunc": "onchaintart", "input": "tell me the current account payable for advanced micro Devices for 2022", "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "AgentExecutor", "type": "custom_agent"}]\'}, {\'role\': \'user\', \'content\': \'[]\'}, {\'role\': \'assistant\', \'content\': \'[]\'}], [{\'role\': \'system\', \'content\': \'[]\'}, {\'role\': \'user\', \'content\': \'[{"input": "tell me the current account payable for advanced micro Devices for 2022", "output": null, "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "LLMChain", "type": "custom_chain"}]\'}, {\'role\': \'assistant\', \'content\': \'[]\'}], [{\'role\': \'system\', \'content\': \'[{"byFunc": "onllmstart", "input": "[\\\\"Answer the following questions as best you can. You have access to the following tools:\\\

In [ ]:
data={"messages": [{"source":"Yash","response": "This is not a valid question", "prompt": "[{\"byFunc\": \"onchaintart\", \"input\": \"hi\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": \"AgentExecutor\", \"type\": \"custom_agent\"}, {\"byFunc\": \"onllmstart\", \"input\": \"[\\\"Answer the following questions as best you can. You have access to the following tools:\\\\n\\\\nintegral: useful for when you need to find integral of a string. It needs input as the string\\\\nBasic_Share_Outstanding: Returns the basic share outstanding from q\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": \"gpt-3.5-turbo-instruct\", \"type\": \"llm\"}, {\"byFunc\": \"onllmend\", \"output\": [null], \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"type\": \"llm\"}, {\"byFunc\": \"on_agentfinish\", \"output\": \"This is not a valid question\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"type\": \"custom_agent\"}]"}, {"role": "user", "content": "[{\"input\": \"hi\", \"output\": null, \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": \"LLMChain\", \"type\": \"custom_chain\"}]"}, {"role": "assistant", "content": "[{\"input\": null, \"output\": \" This is not a valid question\\nFinal Answer: This is not a valid question\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": null, \"type\": \"custom_chain\"}, {\"input\": null, \"output\": \"This is not a valid question\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": null, \"type\": \"custom_chain\"}]"}]}
m=data['messages'][0]["prompt"]
datam={'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction\nWhat are different types of grass?\n\n### Response\n',
 'response': 'There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.',
 'source': 'dolly_hhrlhf'}


In [ ]:
m1={"messages": [{"source":"Yash","response": "This is not a valid question", "prompt": "[{\"byFunc\": \"onchaintart\", \"input\": \"hi\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": \"AgentExecutor\", \"type\": \"custom_agent\"}, {\"byFunc\": \"onllmstart\", \"input\": \"[\\\"Answer the following questions as best you can. You have access to the following tools:\\\\n\\\\nintegral: useful for when you need to find integral of a string. It needs input as the string\\\\nBasic_Share_Outstanding: Returns the basic share outstanding from q\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": \"gpt-3.5-turbo-instruct\", \"type\": \"llm\"}, {\"byFunc\": \"onllmend\", \"output\": [null], \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"type\": \"llm\"}, {\"byFunc\": \"on_agentfinish\", \"output\": \"This is not a valid question\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"type\": \"custom_agent\"}]"}, {"role": "user", "content": "[{\"input\": \"hi\", \"output\": null, \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": \"LLMChain\", \"type\": \"custom_chain\"}]"}, {"role": "assistant", "content": "[{\"input\": null, \"output\": \" This is not a valid question\\nFinal Answer: This is not a valid question\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": null, \"type\": \"custom_chain\"}, {\"input\": null, \"output\": \"This is not a valid question\", \"operatorDescriptiveFeedback\": \"\", \"operatorFeedback\": \"-1\", \"name\": null, \"type\": \"custom_chain\"}]"}]}
m2=


m

SyntaxError: invalid syntax (<ipython-input-11-024dd0c9c2b9>, line 2)

In [ ]:
type(data["messages"])

In [ ]:
print(type(m),m)

In [ ]:
for sample in m:
  print (sample," -")

for sample in datam:
  print (sample)

In [ ]:
b=create_prompt(datam)
type(b)

In [ ]:
a=create_prompt(m)
a

In [ ]:
type(a)

<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.
This is not a valid question[/INST][{"byFunc": "onchaintart", "input": "hi", "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "AgentExecutor", "type": "custom_agent"}, {"byFunc": "onllmstart", "input": "[\"Answer the following questions as best you can. You have access to the following tools:\\n\\nintegral: useful for when you need to find integral of a string. It needs input as the string\\nBasic_Share_Outstanding: Returns the basic share outstanding from q", "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "name": "gpt-3.5-turbo-instruct", "type": "llm"}, {"byFunc": "onllmend", "output": [null], "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "type": "llm"}, {"byFunc": "on_agentfinish", "output": "This is not a valid question", "operatorDescriptiveFeedback": "", "operatorFeedback": "-1", "type": "custom_agent"}]</s>

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
        target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

In [ ]:
print(model)

In [ ]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    print(torch.cuda.device_count())
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "Mixtral_Alpace_v2",
  #num_train_epochs=5,
  max_steps = 1000, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 32,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=10, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2.5e-5,
  bf16=False,
  # lr_scheduler_type='constant',
)

In [ ]:
m

In [ ]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,

  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=args,
  train_dataset=m,
  packing=True,
  # eval_dataset=instruct_tune_dataset["test"]
)